This code is for Example 1 when $A = -100(x^2+y^2, x^2-y^2)$

# This section is to display the graph of A and F

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

# Generate computation domain (-1,1) x (-1,1)
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
# Generate a triangular mesh of mesh-size 0.01, modify maxh value for resutls with h=0.03 and h=0.05
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

# Define vector field A and FEM space for solving the Neumann problem, fix the local polynomial degree at p=3
a =100
A = CF((-a*(x**2 + y**2), -a*(x**2 - y**2)))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

# Define trial-function, test-function, and the variation form 
u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

# the solution field
gfm.vec.data = a.mat.Inverse() * f.vec
# Draw(gfu, mesh);

F = A - grad(gfu)
F0x, F1x = F.Diff(x)
F0y, F1y = F.Diff(y)
Draw(A, mesh,vector=True)
Draw(F, mesh,vector=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm for A and F

In [34]:
sqrt(Integrate(A*A, mesh)), sqrt(Integrate(F*F, mesh))

(178.88543819998287, 70.35843969247841)

# This section is to calculate the eigenpairs for H(A)

In [2]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG

start1 = time.time()
a = 100
A = CF((-a*(x**2 + y**2), -a*(x**2 - y**2)))
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

X = H1(mesh, order=3, complex=True, dirichlet = '.*')
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

# Use FEAST algorithm for computing eigenvalues 
seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 21, center = 45; When h=0.03, radius = 24, center = 50; When h=0.05, radius = 25, center = 51
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius=25, 
                   center=51, 
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
# Plot eigenvectors
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=25, Center=51+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +74.776 +7.725j
SpectralProjNG:   Factorizing at z = +65.695+20.225j
SpectralProjNG:   Factorizing at z = +51.000+25.000j
SpectralProjNG:   Factorizing at z = +36.305+20.225j
SpectralProjNG:   Factorizing at z = +27.224 +7.725j
SpectralProjNG:   Factorizing at z = +27.224 -7.725j
SpectralProjNG:   Factorizing at z = +36.305-20.225j
SpectralProjNG:   Factorizing at z = +51.000-25.000j
SpectralProjNG:   Factorizing at z = +65.695-20.225j
SpectralProjNG:   Factorizing at z = +74.776 -7.725j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [27.06743726 34.34300975 45.77365098 60.06938623 73.08724287 75.24756574
 78.32923333 86.25649047]
   Relative Hausdorff distance from prior iterate: 1.961e+

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the degree of freedom 

In [3]:
X.ndof 

17017

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [4]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

(19.59921071993408+0j) (19.381796935332552+0j)
(31.58120755194521+0j) (31.43313311010435+0j)
(36.88016745237048+0j) (36.734618731143506+0j)
(39.33458914171667+0j) (39.17315258152171+0j)
(22.536560545487728+0j) (22.00173175245896+0j)
(40.34135608483897+0j) (40.14788771744621+0j)


# This section is to calculate the eigenpairs for H(F)

In [6]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG

start1 = time.time()
a = 100
A = CF((-a*(x**2 + y**2), -a*(x**2 - y**2)))

domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N
# fesm.ndof
u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)

Xeig = H1(mesh, order=3, complex = True, dirichlet = '.*')
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

# Make the spectral projector object, radius = 21, center = 45 for all h
seed =1
npts=10
nspan=6
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
P = SpectralProjNG(Xeig,
                a2.mat,
                b2.mat,
                radius=21,
                center=45,
                npts=npts,
                within=within,
                rhoinv=rhoinv,
                quadrule=quadrule,
                inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
# Plot eigenvectors
y1 = Y.gridfun()
Draw(y1)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=21, Center=45+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +64.972 +6.489j
SpectralProjNG:   Factorizing at z = +57.343+16.989j
SpectralProjNG:   Factorizing at z = +45.000+21.000j
SpectralProjNG:   Factorizing at z = +32.657+16.989j
SpectralProjNG:   Factorizing at z = +25.028 +6.489j
SpectralProjNG:   Factorizing at z = +25.028 -6.489j
SpectralProjNG:   Factorizing at z = +32.657-16.989j
SpectralProjNG:   Factorizing at z = +45.000-21.000j
SpectralProjNG:   Factorizing at z = +57.343-16.989j
SpectralProjNG:   Factorizing at z = +64.972 -6.489j

=========== Starting FEAST iterations ===========
Trying with 6 vectors:

 ITERATION 1 with 6 vectors
   Real part of computed eigenvalues:
   [26.58957775 29.94979222 36.33495373 44.5301457  54.73092191 65.85031033]
   Relative Hausdorff distance from prior iterate: 2.222e+98

 ITERATION 2 with 6 v

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \phi$ and $F\phi$

In [32]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

(33.86279130204669+0j) (33.742883827513026+0j)
(26.502209831240194+0j) (26.248030593668215+0j)
(27.605106688924973+0j) (27.252828843856214+0j)
(27.54320023650021+0j) (27.02106185272123+0j)
(27.60503594993768+0j) (26.908753339204793+0j)
(27.535473821535614+0j) (26.662445495388514+0j)
